# Atlantic Marine Birds

This indicator was used in the Southeast Blueprint 2023.
Code written by Rua Mordecai and Amy Keister

In [10]:
import os
import arcpy

In [11]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [12]:
# define final output name
IndicatorFileName = "AtlanticMarineBirds.tif"

In [13]:
# define raster used for cell size, extent, and snapping
MarineRaster= r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [14]:
# define inputs
bird= r"D:\Blueprint\2023\Indicators\AtlanticMarineBirds\zonation\rankmap.tif"
nlcd = r"D:\Landcover\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

In [15]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

### Start analysis

In [16]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

### Bring in zonation results

In [27]:
# take zonation results times 100
out_raster = arcpy.sa.Times(bird, 100); out_raster.save("bird.tif")

In [28]:
# convert to integer
out_raster = arcpy.sa.Int("bird.tif"); out_raster.save("indicator1.tif")

In [8]:
# Reclassify into bins and align with Marine extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify('indicator1.tif', "Value", "0 10 1;11 20 2;21 30 3;31 40 4;41 50 5;51 60 6;61 70 7;71 80 8;81 90 9;91 100 10", "DATA"); out_raster.save("binned.tif")

In [11]:
# Make an water mask in NLCD to change all land pixels to 0
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=MarineRaster, snapRaster=MarineRaster, cellSize=MarineRaster):
    out_raster = arcpy.sa.Reclassify(nlcd, "Value", "0 11 1;12 95 0"); out_raster.save("nlcdWaterMask.tif")

In [12]:
# use mask to turn land values to 0
out_raster = arcpy.sa.Times("nlcdWaterMask.tif", "binned.tif"); out_raster.save(IndicatorFileName)

### Finalize indicator



In [17]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 10:
        return '10 = >90th percentile of importance for marine bird index species (across the full East Coast study area)'
    elif v == 9:
        return '9 = >80th-90th percentile of importance'  
    elif v == 8:
        return '8 = >70th-80th percentile of importance'  
    elif v == 7:
        return '7 = >60th-70th percentile of importance'
    elif v == 6:
        return '6 = >50th-60th percentile of importance'
    elif v == 5:
        return '5 = >40th-50th percentile of importance'
    elif v == 4:
        return '4 = >30th-40th percentile of importance'  
    elif v == 3:
        return '3 = >20th-30th percentile of importance'  
    elif v == 2:
        return '2 = >10th-20th percentile of importance'
    elif v == 1:
        return '1 = ≤10th percentile of importance'
    elif v == 0:
        return '0 = Land'
"""

In [18]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\AtlanticMarineBirds.tif'>

In [15]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 68
	if Value == 9:
		return 72
	if Value == 8:
		return 62
	if Value == 7:
		return 49
	if Value == 6:
		return 38
	if Value == 5:
		return 31
	if Value == 4:
		return 53
	if Value == 3:
		return 109
	if Value == 2:
		return 181
	if Value == 1:
		return 254
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 1
	if Value == 9:
		return 40
	if Value == 8:
		return 74
	if Value == 7:
		return 104
	if Value == 6:
		return 131
	if Value == 5:
		return 158
	if Value == 4:
		return 184
	if Value == 3:
		return 207
	if Value == 2:
		return 222
	if Value == 1:
		return 232
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 84
	if Value == 9:
		return 120
	if Value == 8:
		return 138
	if Value == 7:
		return 142
	if Value == 6:
		return 143
	if Value == 5:
		return 137
	if Value == 4:
		return 121
	if Value == 3:
		return 90
	if Value == 2:
		return 44
	if Value == 1:
		return 37
	else:
		return 255
		
"""

In [17]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'AtlanticMarineBirds.tif'>